In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-15 16:33:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-04-15 16:33:54 (10.9 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [3]:
# tokenizer
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[n] for n in l])


In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [5]:
def get_batch(data):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y
    
xb, yb = get_batch(train_data)

for t in range(block_size):
    print(t)
    context = xb[1, :t+1]
    target = yb[1, t]
    print(context.tolist(), '-', target.tolist())

print(xb.shape)


0
[10] - 0
1
[10, 0] - 19
2
[10, 0, 19] - 56
3
[10, 0, 19, 56] - 53
4
[10, 0, 19, 56, 53] - 39
5
[10, 0, 19, 56, 53, 39] - 52
6
[10, 0, 19, 56, 53, 39, 52] - 2
7
[10, 0, 19, 56, 53, 39, 52, 2] - 1
torch.Size([32, 8])


In [10]:
class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target_idx=None):
        logits = self.token_embedding_table(idx)

        if target_idx is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            # (B, T)
            targets = target_idx.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens=100):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
            

model = BiagramLanguageModel(vocab_size).to(device)
res, loss = model(xb, yb)
print(res.shape, loss)

decode(model.generate(torch.zeros((1, 1), dtype=torch.long).to(device))[0].tolist())


torch.Size([256, 65]) tensor(4.6500, device='cuda:0', grad_fn=<NllLossBackward0>)


"\nMl\nEw&zB.NnYVFFUg.i'zdfkuIJzi\ns VD&RNtuSGRtaWL:ZR?X&BkLEuwodcApLYhAwP;h&TkPYAOd\nzL$ucy,BQ;'aLqx-tULJ"

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for steps in range(5):
    for _ in range(len(train_data) // batch_size):
        xb, yb = get_batch(train_data)
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    print(loss.item())

print(loss.item())

2.4474315643310547
2.5332376956939697
2.41013765335083
2.3916409015655518
2.4504880905151367
2.4504880905151367


In [12]:

decode(model.generate(torch.zeros((1, 1), dtype=torch.long).to(device))[0].tolist())

'\nGENTIV:\nNIOLED wece:\n\nYouered ay w,\nWh, ttad\nSAMy om owone aimak; e patere ma ucle nd ag, I murr cho'